# Writing Formatted Output

## Overview

### Questions

* How do I display status information during a simulation?
* How can I log user-defined quantities?
* How can I write formatted output to a text file?

### Objectives

* Demonstrate **user-defined log quantities**.
* Explain the use of **Table** to display status information during a simulation run.
* Show that **Table** can write to a file.

## Boilerplate code

In [1]:
import hoomd
import datetime

## Define the Simulation

This tutorial executes the Lennard-Jones particle simulation from a previous tutorial. 
See [*Introducing Molecular Dyamics*](../01-Introducing-Molecular-Dynamics/00-index.ipynb) for a complete description of this code.

In [2]:
gpu = hoomd.device.GPU()
sim = hoomd.Simulation(device=gpu)
sim.create_state_from_gsd(filename='../01-Introducing-Molecular-Dynamics/random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

## Formatted output

The **Table** writer formats log quantities in human-readable text and writes them to `stdout` or a file.
**Table** only supports `scalar` and `string` quantities due to the limitations of this format.
This section shows you how to use **Table** to display status information during a simulation run.

## Add quantities to a Logger

The `flags` argument to **Logger** defines the **flags** that it will accept.

In [3]:
logger = hoomd.logging.Logger(flags=['scalar', 'string'])

Log the and simulation `timestep` and `tps` quantities:

In [4]:
logger.add(sim, quantities=['timestep', 'tps'])

[('Simulation', 'timestep'), ('Simulation', 'tps')]

You can also log user-defined quantities using functions, callable class instances, or class properties.
For example, this class computes the estimated time remaining:

In [5]:
class Status():
    def __init__(self, sim):
        self.sim = sim
    
    @property
    def seconds_remaining(self):
        try:
            return (self.sim.final_timestep - self.sim.timestep) / self.sim.tps
        except ZeroDivisionError:
            return 0        
    
    @property
    def etr(self):
        return str(datetime.timedelta(seconds=self.seconds_remaining))


Assign the loggable quantity using the tuple `(object, property_name, flag)`,  where `flag` is the string name of the flag for this quantity. (The tuple for callable objects would be `(callable, flag)`).

In [6]:
status = Status(sim)
logger[('Status', 'etr')] = (status, 'etr', 'string')

Represent the namespace of your user-defined quantity with a tuple of strings - `('Status', 'etr'`) above.
You can use any number of arbitrary strings in the tuple to name your quantity.

## Display quantities with Table

**Table** is a **Writer** that formats the quantities in a **Logger** into a human readable table.
Create one that triggers periodically:

In [7]:
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)

Add it to the simulation:

In [8]:
sim.operations.writers.append(table)

Run the simulation and see the output:

In [9]:
sim.run(100000)

Simulation.timestep  Simulation.tps     Status.etr   
       35000           5221.16583     0:00:18.195170 
       40000           6315.29484     0:00:14.251116 
       45000           7798.08071     0:00:10.900118 
       50000           8834.55757     0:00:09.055349 
       55000           9600.65072     0:00:07.811970 
       60000          10193.53756     0:00:06.867096 
       65000          10661.33219     0:00:06.096799 
       70000          11042.49867     0:00:05.433553 
       75000          11356.92530     0:00:04.842860 
       80000          11622.47944     0:00:04.302008 
       85000          11850.57149     0:00:03.797285 
       90000          12046.98080     0:00:03.320334 
       95000          12217.95819     0:00:02.864636 
      100000          12368.47267     0:00:02.425522 
      105000          12501.93157     0:00:01.999691 
      110000          12619.83273     0:00:01.584807 
      115000          12725.40132     0:00:01.178745 
      120000          12821.

Later in this notebook, you are going to create another **Table Writer**.
Remove `table` now to avoid confusing the two:

In [10]:
sim.operations.writers.remove(table)

## Save Table output to a file

**Table** writes to `stdout` by default.
It can write to a file (or any Python file-like object with `write` and `flush` methods) instead.

Open the file to write:

In [11]:
file = open('log.txt', 'w')

Create a **Table** **Writer** that outputs to this file and add it to the **Simulation**:

In [12]:
table_file = hoomd.write.Table(output=file,
                               trigger=hoomd.trigger.Periodic(period=5000),
                               logger=logger)
sim.operations.writers.append(table_file)

Run the simulation:

In [13]:
sim.run(100000)

You can read the file with standard tools:

In [16]:
!tail log.txt

      185000          14662.99818     0:00:03.068949 
      190000          14669.91888     0:00:02.726668 
      195000          14675.39889     0:00:02.384944 
      200000          14683.10779     0:00:02.043164 
      205000          14686.60521     0:00:01.702231 
      210000          14689.58859     0:00:01.361509 
      215000          14694.45207     0:00:01.020793 
      220000          14698.11772     0:00:00.680359 
      225000          14701.14198     0:00:00.340110 
      230000          14703.20980        0:00:00     


In this section, you have displayed **loggable quantities** during a simulation run and saved them to a text file.
This is the last section in the logging tutorial.

[Previous section](03-Storing-Particle-Shape.ipynb) / [Tutorial index](../README.md)